# CNN torch.ver
# tcmalloc : large alloc error
에러메시지 출력이 아닌 세션 강제종료로 아예 멈춤, 로그 추적하면 해당 메시지를 찾을 수 있지만 메모리 할당 문제라는데.... </br>
=> 나중에 비교

In [0]:
# 1. 모듈 import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import os, numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
batch_size = 100
learning_rate = 0.001
epochs = 100 

In [3]:
# 2. npy 데이터를 불러와서 모델 학습(64 x 64 사이즈의 이미지 약 25000개)
X_train, X_test, y_train, y_test = np.load("/content/drive/My Drive/last.npy", allow_pickle=True)
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')  
print(f'X_test: {X_test.shape}, y_test: {y_test.shape}')

# 데이터 shape [n, c, w, d]로 변경
X_train = np.transpose(X_train, (0, 3, 1, 2))
X_test = np.transpose(X_test, (0, 3, 1, 2))

# data shape 확인
print('x_train shape :', X_train.shape)

X_train: (20802, 64, 64, 3), y_train: (20802, 5)
X_test: (5201, 64, 64, 3), y_test: (5201, 5)
x_train shape : (20802, 3, 64, 64)


In [4]:
# ndarray to tensor
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)
X_train.dtype

torch.uint8

In [0]:
# 4. CNN 모델 구성
# nn.module을 상속받아 구현 및 생성한 객체는 기본적 훈련 모드
# eval() 모드 사용해 추론 모드로 전환 시 dropout, batch_normalization 같은 서로 다른 forward()동작을 하는 모듈에 대해서도 상황에 따라 올바르게 동작
# 단 추론이 끝나면 다시 train()선언해 원래의 훈련모드로 돌아가게 해야함


class cnntorch(nn.Module):

  def __init__(self):
    super(cnntorch, self).__init__()
    self.conv = nn.Sequential(
        # Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # in_channel : RGB | out_channel : 필터의 개수
        nn.Conv2d(3, 32, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 64, 2,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(64, 32, 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25),
        nn.Conv2d(32, 16, 3),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25)
    )

    self.fc_layer = nn.Sequential(
        nn.Linear(16*3*3, 10),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Softmax(),
        nn.Linear(10, 5)
    )
        

  def forward(self, input):
    out = F.relu(self.conv(input))
    out = out.view(batch_size, -1)  # 합성곱 연산과 Linear 연산에서 요구되는 텐서의 형태가 다르다
    out = self.fc_layer(out)
    return out


In [0]:
model = cnntorch()

In [0]:
# 4. 모델 컴파일
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [0]:
# # 5. 모델 학습
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()
  train_output = model(X_train)
  train_loss = criterion(train_output.squeeze(), y_train)
  if epoch % 100 == 0:
    print('Train Loss at {} is {}'.format(epoch, train_loss.item()))
  train_loss.backward()
  optimizer.step()

In [0]:
# 테스트 데이터 검증
correct = 0
total = 0
with torch.no_grad():
  for X_test, y_test in zip(X_test, y_test)
    output = model.forward()
    y_test, output_index = torch.max(output, 1)

    total += label.size(0)
    correct += (output_index == y_test).sum().float()
  print("Accuracy of Test Data: {}".format(100*correct/total))

In [0]:
# loss 그래프
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']

x = range(len(train_loss))
plt.plot(x, train_loss, marker='.', color='red', label='Train loss')
plt.plot(x, val_loss, marker='.', color='blue', label='Val loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss during epochs')
plt.show()

In [0]:
# acc 그래프
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

plt.plot(x, train_acc, marker='.', c='red', label='Train Acc.')
plt.plot(x, val_acc, marker='.', c='blue', label='Val Acc.')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy during epochs')
plt.show()


In [0]:
# Test set으로 모델 평가
score = model.evaluate(X_test, y_test, verbose=0)
print('Training loss:', score[0])
print('Training accuracy: ', score[1])

In [0]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix, classification_report
test_true = np.argmax(y_test, axis=1)
test_pred = np.argmax(model.predict(X_test), axis=1)

cm = confusion_matrix(test_true, test_pred)
print(cm)

In [0]:
# Classification Report
from sklearn.metrics import classification_report
report = classification_report(test_true, test_pred, target_names=os.listdir('/content/drive/My Drive/data_label'))
print(report)